In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_org = pd.read_csv("train.csv")

In [3]:
df_org.sample(5)

,Unnamed: 0,신고번호,신고일자,통관지세관부호,신고인부호,수입자부호,해외거래처부호,특송업체부호,수입통관계획코드,수입신고구분코드,...,운송수단유형코드,반입보세구역부호,HS10단위부호,적출국가코드,원산지국가코드,관세율구분코드,관세율,검사결과코드,우범여부,핵심적발
32967,32967,14516592,2020-04-17,40,1QC72,LCKP31K,8RTOWC1,NaN,D,B,...,40,4077001,9103100000,SG,CH,A,8.0,A,0,0
41329,41329,16599915,2020-05-19,10,QUP4L,5N2GMZF,NaN,NaN,D,B,...,10,3077016,8471300000,US,US,C,0.0,A,0,0
10399,10399,41488748,2020-02-01,40,QUP4L,GWT6N1I,NGNARJN,NaN,F,B,...,40,1011182,3304999000,AE,KR,A,8.0,C2,1,2
30895,30895,31111781,2020-04-09,40,Z3EO2,OH1YU0O,2PYLVNK,NaN,B,B,...,40,2010360,7410110000,JP,JP,A,8.0,A,0,0
52733,52733,86907736,2020-07-01,20,0BW53,XE3K18U,DENR8NT,6RJ8HT,F,B,...,40,2077002,3304991000,US,US,C,6.5,A,0,0


In [4]:
del df_org['검사결과코드']
del df_org['핵심적발']
del df_org['신고번호']
del df_org['수입자부호']
del df_org['해외거래처부호']
del df_org['반입보세구역부호']
del df_org['HS10단위부호']


In [5]:
# 범주형 변수 지정
discrete_columns = ['신고일자',  '통관지세관부호',
       '신고인부호',  '특송업체부호', '수입통관계획코드', 
       '수입신고구분코드', '수입거래구분코드',
       '수입종류코드', '징수형태코드', 
       '운송수단유형코드',  
        '적출국가코드', '원산지국가코드', '관세율구분코드']

# 범주형 변수들을 문자열로 지정 ('object' -> 'string')
for var in discrete_columns:
    df_org[var] = df_org[var].astype(str)

In [6]:
# 수치형 변수 지정
numeric_columns = ['신고중량(KG)', '과세가격원화금액', '관세율', '우범여부']

# 수치형 변수를 자연로그로 변환
for var in ['신고중량(KG)', '과세가격원화금액']:
    df_org[var] = df_org[var].apply(lambda x: np.log1p(x))

In [7]:
df_org.head(5)

,Unnamed: 0,신고일자,통관지세관부호,신고인부호,특송업체부호,수입통관계획코드,수입신고구분코드,수입거래구분코드,수입종류코드,징수형태코드,신고중량(KG),과세가격원화금액,운송수단유형코드,적출국가코드,원산지국가코드,관세율구분코드,관세율,우범여부
0,0,2020-01-01,121,2O5A2,TQ18AK,D,B,15,23,43,4.844187,8.593963,10,US,US,A,8.0,1
1,1,2020-01-01,30,305K5,nan,C,B,11,21,11,10.303820,13.258812,40,US,US,A,8.0,0
2,2,2020-01-01,20,CGMT6,nan,D,B,11,21,18,10.067242,10.862759,40,US,US,A,10.0,0
3,3,2020-01-01,40,QWUTG,nan,C,B,94,21,43,9.429564,14.388527,40,CN,CN,A,8.0,0
4,4,2020-01-01,30,0X1CO,nan,C,B,11,21,43,9.661015,15.987683,40,CN,CN,FCN1,5.2,1


# 원본 데이터셋

In [8]:
df_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76837 entries, 0 to 76836
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  76837 non-null  int64  
 1   신고일자        76837 non-null  object 
 2   통관지세관부호     76837 non-null  object 
 3   신고인부호       76837 non-null  object 
 4   특송업체부호      76837 non-null  object 
 5   수입통관계획코드    76837 non-null  object 
 6   수입신고구분코드    76837 non-null  object 
 7   수입거래구분코드    76837 non-null  object 
 8   수입종류코드      76837 non-null  object 
 9   징수형태코드      76837 non-null  object 
 10  신고중량(KG)    76837 non-null  float64
 11  과세가격원화금액    76837 non-null  float64
 12  운송수단유형코드    76837 non-null  object 
 13  적출국가코드      76837 non-null  object 
 14  원산지국가코드     76837 non-null  object 
 15  관세율구분코드     76837 non-null  object 
 16  관세율         76837 non-null  float64
 17  우범여부        76837 non-null  int64  
dtypes: float64(3), int64(2), object(13)
memory usage: 10.6+ MB


# 훈련/시험 데이터셋 분할
# 이후 레이블 인코딩 진행

In [9]:
df_train = df_org.iloc[:int(len(df_org)*0.9), :]
df_test = df_org.iloc[int(len(df_org)*0.9):, :]

In [10]:
# 범주형 변수 전처리 1단계: label encoding --> 각 범주형 변수가 갖는 클래스에 고유의 식별번호 부여
# 예) 수입자 상호: AAABB -> 1, 가나다라 -> 2, ...

label_encoding_ref = {}
for var in discrete_columns:
    label_encoding_ref[var] = {code: i+1 for i, code in enumerate(df_train[var].unique())} # 0 padding for unseen data in test
    #print(label_encoding_ref[var])
    df_train[var] = [label_encoding_ref[var][x] for x in df_train[var]]
    df_test[var] = [label_encoding_ref[var].get(x,0) for x in df_test[var]]

df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

<ipython-input-10-8ef4b9db707e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[var] = [label_encoding_ref[var][x] for x in df_train[var]]
<ipython-input-10-8ef4b9db707e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[var] = [label_encoding_ref[var].get(x,0) for x in df_test[var]]


In [11]:
df_train, df_test

(       Unnamed: 0  신고일자  통관지세관부호  신고인부호  특송업체부호  수입통관계획코드  수입신고구분코드  수입거래구분코드  \
 0               0     1        1      1       1         1         1         1   
 1               1     1        2      2       2         2         1         2   
 2               2     1        3      3       2         1         1         2   
 3               3     1        4      4       2         2         1         3   
 4               4     1        2      5       2         2         1         2   
 ...           ...   ...      ...    ...     ...       ...       ...       ...   
 69148       69148   243        2     99      36         1         1         1   
 69149       69149   243        4     24       2         1         1         2   
 69150       69150   243        4     24       2         1         1         2   
 69151       69151   243        5     20       2         1         1         2   
 69152       69152   243        4    354       2         3         1         2   
 
        수입종류코드

In [12]:
df_label_encoded = pd.concat([df_train, df_test])
df_label_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76837 entries, 0 to 7683
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  76837 non-null  int64  
 1   신고일자        76837 non-null  int64  
 2   통관지세관부호     76837 non-null  int64  
 3   신고인부호       76837 non-null  int64  
 4   특송업체부호      76837 non-null  int64  
 5   수입통관계획코드    76837 non-null  int64  
 6   수입신고구분코드    76837 non-null  int64  
 7   수입거래구분코드    76837 non-null  int64  
 8   수입종류코드      76837 non-null  int64  
 9   징수형태코드      76837 non-null  int64  
 10  신고중량(KG)    76837 non-null  float64
 11  과세가격원화금액    76837 non-null  float64
 12  운송수단유형코드    76837 non-null  int64  
 13  적출국가코드      76837 non-null  int64  
 14  원산지국가코드     76837 non-null  int64  
 15  관세율구분코드     76837 non-null  int64  
 16  관세율         76837 non-null  float64
 17  우범여부        76837 non-null  int64  
dtypes: float64(3), int64(15)
memory usage: 11.1 MB


# 원핫인코딩 진행할 때 데이터셋 프로세스

# * 원핫 진행할 때 이 셀로 시작 *

In [13]:
df_train = df_label_encoded.iloc[:int(len(df_org)*0.9), :]
df_test = df_label_encoded.iloc[int(len(df_org)*0.9):, :]

df_train.info(), df_test.info()

from sklearn.preprocessing import OneHotEncoder
# 원핫인코더 활성화
xgbohe = OneHotEncoder(categories="auto", handle_unknown='ignore')
# 훈련데이터: 범주형 변수만 분리하여 원핫인코딩 적용
df_train_cat = xgbohe.fit_transform(df_train[discrete_columns])
df_train_cat = pd.DataFrame(df_train_cat.toarray(), 
                                columns = xgbohe.get_feature_names(discrete_columns))
# 수치형 변수 분리
df_train_num = df_train[numeric_columns]

# 원핫인코딩된 범주형 변수와 수치형 변수 재결합
df_train = pd.concat([df_train_num, df_train_cat], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69153 entries, 0 to 69152
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  69153 non-null  int64  
 1   신고일자        69153 non-null  int64  
 2   통관지세관부호     69153 non-null  int64  
 3   신고인부호       69153 non-null  int64  
 4   특송업체부호      69153 non-null  int64  
 5   수입통관계획코드    69153 non-null  int64  
 6   수입신고구분코드    69153 non-null  int64  
 7   수입거래구분코드    69153 non-null  int64  
 8   수입종류코드      69153 non-null  int64  
 9   징수형태코드      69153 non-null  int64  
 10  신고중량(KG)    69153 non-null  float64
 11  과세가격원화금액    69153 non-null  float64
 12  운송수단유형코드    69153 non-null  int64  
 13  적출국가코드      69153 non-null  int64  
 14  원산지국가코드     69153 non-null  int64  
 15  관세율구분코드     69153 non-null  int64  
 16  관세율         69153 non-null  float64
 17  우범여부        69153 non-null  int64  
dtypes: float64(3), int64(15)
memory usage: 10.0 MB
<class 'pandas.core

In [14]:
# 테스트데이터: 범주형 변수만 분리하여 원핫인코딩 적용
df_test_cat = xgbohe.transform(df_test[discrete_columns])
df_test_cat = pd.DataFrame(df_test_cat.toarray(), 
                               columns = xgbohe.get_feature_names(discrete_columns))
# 수치형 변수 분리
df_test_num = df_test[numeric_columns]
# 원핫인코딩된 범주형 변수와 수치형 변수 재결합
df_test = pd.concat([df_test_num, df_test_cat], axis=1)

In [15]:
df_train.shape

(69153, 1610)

# 원핫 혹은 레이블 인코딩 완료
# 완전한 데이터셋 구성 후 분리

In [16]:
df_train_x = df_train
df_train_y = df_train_x.pop("우범여부")

In [17]:
df_test_x = df_test
df_test_y = df_test_x.pop("우범여부")

In [18]:
df_train_x, df_train_y

(        신고중량(KG)   과세가격원화금액   관세율  신고일자_1  신고일자_2  신고일자_3  신고일자_4  신고일자_5  \
 0       4.844187   8.593963   8.0     1.0     0.0     0.0     0.0     0.0   
 1      10.303820  13.258812   8.0     1.0     0.0     0.0     0.0     0.0   
 2      10.067242  10.862759  10.0     1.0     0.0     0.0     0.0     0.0   
 3       9.429564  14.388527   8.0     1.0     0.0     0.0     0.0     0.0   
 4       9.661015  15.987683   5.2     1.0     0.0     0.0     0.0     0.0   
 ...          ...        ...   ...     ...     ...     ...     ...     ...   
 69148   7.792390  12.591555   0.0     0.0     0.0     0.0     0.0     0.0   
 69149   8.601589  12.955920  13.0     0.0     0.0     0.0     0.0     0.0   
 69150   8.892460  13.705144   0.0     0.0     0.0     0.0     0.0     0.0   
 69151   9.107255   9.934120   6.5     0.0     0.0     0.0     0.0     0.0   
 69152   9.045560  14.363811   8.0     0.0     0.0     0.0     0.0     0.0   
 
        신고일자_6  신고일자_7  ...  관세율구분코드_26  관세율구분코드_27  관세율구분코드_2

# Feature Selection Sampling Test

In [19]:
X, y = df_train_x.iloc[:1000, 1:], df_train_y.iloc[:1000]
# Sampling 1000 - 신고번호 제외

# SVM - RFECV Feature Selection

In [20]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring is proportional to the number of correct
# classifications

min_features_to_select = 1  # Minimum number of features to consider
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='accuracy',
              min_features_to_select=min_features_to_select)
rfecv.fit(X, y)
print("Optimal number of features : %d" % rfecv.n_features_)

KeyboardInterrupt: 

In [ ]:
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

In [ ]:
rfecv.get_support([""])

# Tree Based Feature Selection

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)

In [ ]:
clf.feature_importances_.argsort()

In [ ]:
X_new.shape

# RFE - RandomForestRegressor Feature Selection

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

select = RFE(estimator=RandomForestRegressor(n_estimators=10)) # median, mean, 0.2

select_x = select.fit_transform(X, y)
select_x.shape

# chi2 - KBest Feature Selection

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

df_chi2 = SelectKBest(chi2, 10).fit_transform(X, y)
df_chi2.shape

# Imbalanced Sampling
# * SMOTE *

In [ ]:
from imblearn.over_sampling import *

In [ ]:
smote = SMOTE(random_state=11)
df_train_over, df_train_over_y = smote.fit_resample(df_select_new_train, df_train_y)
df_train_over.info(), df_train_over_y

---
---

In [ ]:
from collections import Counter
cnt_train = Counter(df_train_y)
cnt_test = Counter(df_test_y)

print(f"훈련  데이터 비우범건수: {cnt_train[0]}, 훈련  데이터 우범건수: {cnt_train[1]}")
print(f"테스트데이터 비우범건수: {cnt_test[0]}, 테스트데이터 우범건수: {cnt_test[1]}")

# 모델 생성 / 학습 / 평가

In [ ]:
# XGBoost 모델 생성
from xgboost import XGBClassifier
from sklearn.metrics import f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
xgb_clf = XGBClassifier(n_estimators=10, max_depth=4,n_jobs=-1)
eval_set = [(df_train_over, df_train_over_y), (df_select_new_test, df_test_y)]
xgb_clf.fit(df_train_over, df_train_over_y, eval_metric=["logloss"], eval_set=eval_set, verbose=True)

In [ ]:
# 모델 성능 시각화

from matplotlib import pyplot
# retrieve performance metrics
results = xgb_clf.evals_result()
epochs = len(results['validation_0']['logloss'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')

In [ ]:
# evaluate xgboost model
print("------Evaluating xgboost model------")
# Predict
test_pred = xgb_clf.predict_proba(df_select_new_test)[:,1]
# Calculate auc
xgb_auc = roc_auc_score(df_test_y, test_pred)
print(xgb_auc)

In [ ]:
def inspection_performance(predicted_fraud, test_fraud):
        
    Inspect_Rate = []
    Precision=[]
    Recall=[]
    
    for i in range(0,101,1):
        
        threshold = np.percentile(predicted_fraud, i)
        # Precision = number of frauds / number of inspection
        precision = np.mean(test_fraud[predicted_fraud >= threshold])
        # Recall = number of inspected frauds / number of frauds
        recall = sum(test_fraud[predicted_fraud >= threshold])/sum(test_fraud)
        # Save values
        Inspect_Rate.append(100-i)
        Precision.append(precision)
        Recall.append(recall)
        
    
    compiled_conf_matrix = pd.DataFrame({
        
        'Inspect_Rate':Inspect_Rate,
        'Precision':Precision,
        'Recall':Recall
    })

    return compiled_conf_matrix

In [ ]:
basic_performance = inspection_performance(test_pred, df_test_y.astype(float))

In [ ]:
# 검사율 1~10% 지정 시 Precision 및 Recall 분석
# Precision (적중률) = (검사선별된 우범건수)/(검사선별 건수)
# Recall (적발률) = (검사선별된 우범건수) / (전체 우범건수)
basic_performance.iloc[range(99,89,-1),:]

In [ ]:
data = pd.melt(basic_performance, 
               id_vars = ['Inspect_Rate'],
               value_vars = ['Recall','Precision'])

sns.relplot(data=data,
            kind='line',
            x="Inspect_Rate", 
            y="value", 
            hue='variable',
            col="variable")

In [ ]:
# 모델에 기여도가 높은 변수 시각화
from xgboost import plot_importance
plt.rcParams["font.family"] = 'Malgun Gothic'
plt.rcParams["figure.figsize"] = (15,10)
plot_importance(xgb_clf, max_num_features=30)
plt.show()